In [384]:
import subprocess as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
import xml.etree.ElementTree as ET
import re
from operator import itemgetter
import csv
import os
import numpy as np
from lightning.ranking import PRank

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/joy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/joy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Read json data set

In [3]:
with open('/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/document_passages.json') as json_file:
    corpus = json.load(json_file)

In [4]:
len(corpus)

863

### Convert corpus to passage list and generate row dict

In [5]:
corpus_list = []
row_dict = {}
cnt = 0
for k1 in corpus:
    p = corpus[k1]
    for k2 in p:
        corpus_list.append(p[k2])
        row_dict[k1+'-'+k2] = cnt
        cnt = cnt + 1

In [52]:
corpus_list

['Townspeople who lived in chartered towns were burghers, as opposed to serfs who lived in villages. Towns were often "free", in the sense that they were directly protected by the king or emperor, and were not part of a feudal fief. [citation needed]\n\nToday the process for granting charters is determined by the type of government of the state in question. In monarchies, charters are still often a royal charter given by the Crown or the state authorities acting on behalf of the Crown. In federations, the granting of charters may be within the jurisdiction of the lower level of government such as a state or province. [citation needed]\n\nIn Brazil, municipal corporations are called municipios and are created by means of local legislation at the state level, or after passing a referendum vote of the affected population.',
 'A municipal corporation is the legal term for a local governing body, including cities, counties, towns, townships, charter townships, villages, and boroughs. Munici

### Preprocess corpus

In [19]:
def pre_pro(sentence_list):
    lemmatizer = WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    more = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]
    new_stop = stopwords + more
    q_tokens_f=[]
    for i in range(len(sentence_list)):
        q_tokens = [lemmatizer.lemmatize(word) for word in word_tokenize(sentence_list[i]) if word not in new_stop]
        q_tokens_f.append(' '.join(q_tokens))
    return q_tokens_f

In [613]:
def pre_pro_for_dict(sentence_dict):
    lemmatizer = WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    more = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]
    print(len(more))
    new_stop = stopwords + more
    q_tokens_f=[]
    for doc_k in corpus:
        passage = corpus[doc_k]
        for p_k in passage:
            q_tokens = [lemmatizer.lemmatize(word) for word in word_tokenize(passage[p_k]) if word not in new_stop]
            sentence_dict[doc_k][p_k] = ' '.join(q_tokens)

In [615]:
more = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

In [616]:
len(more)

319

In [614]:
corpus_list_prec = pre_pro(corpus_list)
print(corpus_list[0])
print(type(corpus_list_prec[0]))
print(corpus_list_prec[0])

KeyboardInterrupt: 

In [120]:
corpus_list_prec[0]

"Townspeople lived chartered town burgher , opposed serf lived village . Towns `` free '' , sense directly protected king emperor , feudal fief . [ citation needed ] Today process granting charter determined type government state question . In monarchy , charter royal charter given Crown state authority acting behalf Crown . In federation , granting charter jurisdiction lower level government state province . [ citation needed ] In Brazil , municipal corporation called municipios created mean local legislation state level , passing referendum vote affected population ."

### Generate matrix of token counts

In [23]:
vectorizer = CountVectorizer()
TF_mat = vectorizer.fit_transform(corpus_list_prec)
word_names = vectorizer.get_feature_names()

In [45]:
TF_mat.shape, range(TF_mat.shape[1]), type(TF_mat), TF_mat[0]

((50612, 130484),
 range(0, 130484),
 scipy.sparse.csr.csr_matrix,
 <1x130484 sparse matrix of type '<class 'numpy.int64'>'
 	with 55 stored elements in Compressed Sparse Row format>)

### Create column dict

In [25]:
column_dict = dict(zip(word_names, range(TF_mat.shape[1])))

In [37]:
column_dict['what']

126810

## Ranking
### Get queries with preprocessing

In [488]:
path = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/test.tsv'
path_p = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/document_passages.json'
question = []
passage = []
# question
counter = 0
line_ori = []
QID = []
rel_doc_id = []
with open(path,'r') as f:
    while 1:
        line = f.readline()
        line_ori.append(line)
        counter += 1
        if len(line) != 0:
            # delete all characters
            question.append(re.sub('\W'," ",line.split('\t')[1].strip('?')))
            QID.append(line.split('\t')[0])
            rel_doc_id.append(line.split('\t')[2])
        if not line:
            break
        pass #do something
# the first row is all for titles, delete that one
del question[0]
del QID[0]
del rel_doc_id[0]
print(type(question))
print(len(question))
print(question[415])
print(rel_doc_id)

<class 'list'>
416
Why are drainage basins important to the ecology
['543', '376', '699', '131', '441', '646', '474', '847', '337', '565', '527', '260', '452', '424', '180', '770', '373', '451', '142', '616', '573', '243', '361', '323', '26', '670', '749', '544', '158', '313', '334', '276', '382', '861', '386', '454', '387', '679', '520', '300', '174', '648', '598', '136', '430', '567', '75', '136', '774', '289', '579', '461', '595', '531', '60', '814', '340', '515', '654', '836', '196', '163', '49', '410', '282', '384', '283', '99', '760', '779', '675', '303', '484', '255', '698', '345', '370', '585', '678', '220', '788', '647', '636', '657', '751', '489', '134', '536', '267', '22', '71', '13', '636', '1', '801', '808', '103', '511', '807', '260', '130', '269', '465', '721', '347', '499', '515', '719', '236', '255', '96', '95', '510', '850', '643', '365', '227', '530', '712', '818', '20', '564', '667', '485', '495', '590', '537', '95', '219', '624', '758', '305', '592', '764', '340', 

In [368]:
question_prec = pre_pro(question)
len(question_prec)

3332

In [363]:
question[0], type(question_prec), question_prec[0]

('What is the role of conversionism in Evangelicalism',
 list,
 'What role conversionism Evangelicalism')

In [489]:
QID_dict = dict(zip(question_prec, QID))
rel_doc_id_dict = dict(zip(QID, rel_doc_id))
QID_dict

{'What role conversionism Evangelicalism': '449',
 'How assault Bastille year Revolution ultimately culminate capture execution Louis XVI January 1793': '1140',
 'What prehistory Albania': '782',
 'What significance Bulgaria ending World War I': '1421',
 'What rationale support Common Era': '3738',
 'What characterized Indonesia foreign relation New Order era': '844',
 'How violent Pacific Ocean': '1201',
 'Why Sub Saharan Africa considered paradoxical birth rate': '4076',
 'What effect Napoleonic Wars Britain empire': '404',
 'According economist HDI misguided statistic': '3915',
 'How boundary Appalachians vary according different people': '3675',
 'What influence native people Colorado history state': '104',
 'Hydroelectric used today': '1214',
 'What recommdations OECD working group headed Japanese ambassador make 2003': '1402',
 'Why power decentralized numerous period early Byzantium history': '3551',
 'Why Sadat economic policy actually end benefitting rich': '3981',
 'Describe 

### Extract word count and rank the answer

In [370]:
ranking_score = {}

In [371]:
question_prec[0].split()[1].lower()

'role'

In [382]:
QID_dict

{'What role conversionism Evangelicalism': '3086',
 'How assault Bastille year Revolution ultimately culminate capture execution Louis XVI January 1793': '195',
 'What prehistory Albania': '557',
 'What significance Bulgaria ending World War I': '1508',
 'What rationale support Common Era': '956',
 'What characterized Indonesia foreign relation New Order era': '1993',
 'How violent Pacific Ocean': '2260',
 'Why Sub Saharan Africa considered paradoxical birth rate': '2678',
 'What effect Napoleonic Wars Britain empire': '2165',
 'According economist HDI misguided statistic': '2079',
 'How boundary Appalachians vary according different people': '1287',
 'What influence native people Colorado history state': '2958',
 'Hydroelectric used today': '4002',
 'What recommdations OECD working group headed Japanese ambassador make 2003': '1634',
 'Why power decentralized numerous period early Byzantium history': '2610',
 'Why Sadat economic policy actually end benefitting rich': '3381',
 'Describ

In [373]:
# for all questions
for q in question_prec:
    ranking_score[QID_dict[q]] = {}
    # for all passages
    for p in row_dict:
        if rel_doc_id_dict[QID_dict[q]] in p:
            tmp_score = 0
            # for all words in the query
            for word in q.split():
                if word.lower() in column_dict:
                    # sum up term frequency of every word
                    tmp_score = tmp_score + TF_mat[row_dict[p], column_dict[word.lower()]]
            ranking_score[QID_dict[q]][p] = tmp_score
    print('Finished query: ' + QID_dict[q])

Finished query: 3086
Finished query: 195
Finished query: 557
Finished query: 1508
Finished query: 956
Finished query: 1993
Finished query: 2260
Finished query: 2678
Finished query: 2165
Finished query: 2079
Finished query: 1287
Finished query: 2958
Finished query: 4002
Finished query: 1634
Finished query: 2610
Finished query: 3381
Finished query: 3534
Finished query: 511
Finished query: 1824
Finished query: 25
Finished query: 75
Finished query: 1435
Finished query: 1545
Finished query: 1130
Finished query: 1372
Finished query: 458
Finished query: 2450
Finished query: 3203
Finished query: 1831
Finished query: 4033
Finished query: 3565
Finished query: 2938
Finished query: 2822
Finished query: 3295
Finished query: 1205
Finished query: 1705
Finished query: 3318
Finished query: 2497
Finished query: 892
Finished query: 3200
Finished query: 861
Finished query: 3163
Finished query: 3047
Finished query: 3468
Finished query: 3471
Finished query: 774
Finished query: 1317
Finished query: 3038
Fini

Finished query: 1771
Finished query: 176
Finished query: 3088
Finished query: 572
Finished query: 3828
Finished query: 1764
Finished query: 284
Finished query: 376
Finished query: 3260
Finished query: 1628
Finished query: 427
Finished query: 1776
Finished query: 159
Finished query: 837
Finished query: 1332
Finished query: 134
Finished query: 3831
Finished query: 3380
Finished query: 926
Finished query: 3875
Finished query: 3891
Finished query: 1843
Finished query: 3755
Finished query: 579
Finished query: 3248
Finished query: 3294
Finished query: 3462
Finished query: 4006
Finished query: 838
Finished query: 276
Finished query: 1315
Finished query: 1797
Finished query: 4005
Finished query: 1042
Finished query: 2924
Finished query: 4051
Finished query: 295
Finished query: 3536
Finished query: 531
Finished query: 2411
Finished query: 2835
Finished query: 434
Finished query: 918
Finished query: 3297
Finished query: 423
Finished query: 1077
Finished query: 2572
Finished query: 554
Finished q

Finished query: 2214
Finished query: 704
Finished query: 854
Finished query: 3437
Finished query: 2614
Finished query: 4101
Finished query: 2823
Finished query: 2330
Finished query: 1569
Finished query: 855
Finished query: 546
Finished query: 613
Finished query: 4003
Finished query: 88
Finished query: 537
Finished query: 1732
Finished query: 2800
Finished query: 3697
Finished query: 1399
Finished query: 2799
Finished query: 514
Finished query: 1719
Finished query: 4173
Finished query: 13
Finished query: 1281
Finished query: 3581
Finished query: 2698
Finished query: 278
Finished query: 2104
Finished query: 443
Finished query: 623
Finished query: 1068
Finished query: 365
Finished query: 2733
Finished query: 581
Finished query: 2703
Finished query: 1519
Finished query: 408
Finished query: 1620
Finished query: 2287
Finished query: 3344
Finished query: 3476
Finished query: 792
Finished query: 3049
Finished query: 2752
Finished query: 1482
Finished query: 1789
Finished query: 2634
Finished q

Finished query: 80
Finished query: 3187
Finished query: 1576
Finished query: 1204
Finished query: 162
Finished query: 1352
Finished query: 3957
Finished query: 128
Finished query: 2094
Finished query: 2763
Finished query: 2997
Finished query: 1483
Finished query: 3154
Finished query: 3085
Finished query: 181
Finished query: 3585
Finished query: 2215
Finished query: 485
Finished query: 945
Finished query: 1344
Finished query: 2245
Finished query: 1095
Finished query: 2309
Finished query: 3602
Finished query: 4117
Finished query: 2451
Finished query: 2790
Finished query: 222
Finished query: 1443
Finished query: 4008
Finished query: 166
Finished query: 2787
Finished query: 1593
Finished query: 529
Finished query: 3966
Finished query: 3291
Finished query: 2927
Finished query: 3757
Finished query: 3061
Finished query: 1649
Finished query: 2187
Finished query: 3496
Finished query: 647
Finished query: 2882
Finished query: 2103
Finished query: 3267
Finished query: 3659
Finished query: 3609
Fin

Finished query: 52
Finished query: 508
Finished query: 2960
Finished query: 2573
Finished query: 3013
Finished query: 1826
Finished query: 2263
Finished query: 2513
Finished query: 95
Finished query: 2738
Finished query: 1927
Finished query: 3611
Finished query: 4107
Finished query: 543
Finished query: 839
Finished query: 3173
Finished query: 2932
Finished query: 1514
Finished query: 929
Finished query: 3516
Finished query: 142
Finished query: 2381
Finished query: 4153
Finished query: 3889
Finished query: 3278
Finished query: 1949
Finished query: 2861
Finished query: 3603
Finished query: 2962
Finished query: 2612
Finished query: 1431
Finished query: 2936
Finished query: 374
Finished query: 3296
Finished query: 1696
Finished query: 975
Finished query: 1730
Finished query: 1091
Finished query: 4103
Finished query: 1699
Finished query: 1543
Finished query: 1143
Finished query: 255
Finished query: 2569
Finished query: 660
Finished query: 2014
Finished query: 3450
Finished query: 2860
Finis

Finished query: 2051
Finished query: 3413
Finished query: 1356
Finished query: 2130
Finished query: 963
Finished query: 2683
Finished query: 1410
Finished query: 941
Finished query: 940
Finished query: 1918
Finished query: 3937
Finished query: 3529
Finished query: 1165
Finished query: 670
Finished query: 1610
Finished query: 3293
Finished query: 3626
Finished query: 452
Finished query: 1019
Finished query: 70
Finished query: 1216
Finished query: 3401
Finished query: 5
Finished query: 2085
Finished query: 1695
Finished query: 347
Finished query: 1867
Finished query: 2128
Finished query: 3965
Finished query: 3680
Finished query: 4106
Finished query: 2735
Finished query: 2122
Finished query: 973
Finished query: 1060
Finished query: 3137
Finished query: 2538
Finished query: 2326
Finished query: 1911
Finished query: 2364
Finished query: 1271
Finished query: 1235
Finished query: 130
Finished query: 2389
Finished query: 1505
Finished query: 585
Finished query: 2169
Finished query: 1590
Finish

Finished query: 2193
Finished query: 1736
Finished query: 205
Finished query: 293
Finished query: 2209
Finished query: 1249
Finished query: 1137
Finished query: 1003
Finished query: 2854
Finished query: 1208
Finished query: 3500
Finished query: 3152
Finished query: 340
Finished query: 1310
Finished query: 2339
Finished query: 1547
Finished query: 2430
Finished query: 2030
Finished query: 3643
Finished query: 1161
Finished query: 3339
Finished query: 2257
Finished query: 3204
Finished query: 794
Finished query: 43
Finished query: 2793
Finished query: 2055
Finished query: 20
Finished query: 2833
Finished query: 3071
Finished query: 2175
Finished query: 2754
Finished query: 952
Finished query: 2795
Finished query: 2662
Finished query: 4044
Finished query: 4157
Finished query: 3465
Finished query: 2020
Finished query: 721
Finished query: 1846
Finished query: 2409
Finished query: 3949
Finished query: 575
Finished query: 106
Finished query: 2916
Finished query: 1159
Finished query: 1101
Fini

Finished query: 1690
Finished query: 3570
Finished query: 2333
Finished query: 356
Finished query: 953
Finished query: 3415
Finished query: 2331
Finished query: 584
Finished query: 3712
Finished query: 4032
Finished query: 1761
Finished query: 2695
Finished query: 3333
Finished query: 3988
Finished query: 1416
Finished query: 3634
Finished query: 3541
Finished query: 981
Finished query: 2986
Finished query: 1939
Finished query: 1457
Finished query: 519
Finished query: 2878
Finished query: 2640
Finished query: 2757
Finished query: 3777
Finished query: 1484
Finished query: 3141
Finished query: 3188
Finished query: 2933
Finished query: 1914
Finished query: 2378
Finished query: 2498
Finished query: 2804
Finished query: 2230
Finished query: 153
Finished query: 3901
Finished query: 173
Finished query: 819
Finished query: 2601
Finished query: 1947
Finished query: 3502
Finished query: 313
Finished query: 2626
Finished query: 947
Finished query: 310
Finished query: 3408
Finished query: 65
Finis

Finished query: 710
Finished query: 3592
Finished query: 143
Finished query: 3191
Finished query: 2490
Finished query: 3315
Finished query: 2742
Finished query: 3483
Finished query: 2407
Finished query: 3871
Finished query: 3405
Finished query: 2203
Finished query: 2292
Finished query: 3080
Finished query: 111
Finished query: 3635
Finished query: 4048
Finished query: 974
Finished query: 2984
Finished query: 124
Finished query: 1170
Finished query: 1075
Finished query: 359
Finished query: 913
Finished query: 3312
Finished query: 461
Finished query: 3808
Finished query: 1328
Finished query: 319
Finished query: 2298
Finished query: 691
Finished query: 3677
Finished query: 1923
Finished query: 3504
Finished query: 2808
Finished query: 1910
Finished query: 3135
Finished query: 3097
Finished query: 2602
Finished query: 882
Finished query: 192
Finished query: 1237
Finished query: 2512
Finished query: 1553
Finished query: 3857
Finished query: 3284
Finished query: 64
Finished query: 625
Finishe

In [66]:
ranking_score

{'449': {'543-56': 5,
  '543-77': 2,
  '543-54': 4,
  '543-42': 3,
  '543-48': 0,
  '543-43': 1,
  '543-60': 1,
  '543-61': 4,
  '543-62': 5,
  '543-63': 1,
  '543-64': 4,
  '543-49': 7,
  '543-66': 2,
  '543-67': 2,
  '543-68': 1,
  '543-69': 2,
  '543-52': 3,
  '543-53': 3,
  '543-24': 0,
  '543-25': 3,
  '543-26': 2,
  '543-27': 2,
  '543-20': 0,
  '543-21': 2,
  '543-22': 4,
  '543-23': 3,
  '543-46': 3,
  '543-47': 0,
  '543-44': 2,
  '543-45': 2,
  '543-28': 1,
  '543-29': 4,
  '543-40': 1,
  '543-41': 2,
  '543-1': 3,
  '543-0': 5,
  '543-3': 4,
  '543-2': 4,
  '543-5': 2,
  '543-4': 1,
  '543-7': 1,
  '543-6': 3,
  '543-9': 3,
  '543-8': 2,
  '543-78': 4,
  '543-51': 4,
  '543-39': 3,
  '543-65': 0,
  '543-76': 3,
  '543-75': 1,
  '543-38': 4,
  '543-73': 3,
  '543-72': 2,
  '543-71': 2,
  '543-70': 2,
  '543-59': 2,
  '543-79': 1,
  '543-58': 3,
  '543-11': 2,
  '543-10': 2,
  '543-13': 1,
  '543-12': 2,
  '543-15': 0,
  '543-14': 2,
  '543-17': 1,
  '543-16': 1,
  '543-19': 0

###  Convert rank to list for writing to file and sort it

In [374]:
ranking_list = []
for q_id in ranking_score:
    for p_id in ranking_score[q_id]:
        ranking_list.append([q_id, '00', p_id, 0, ranking_score[q_id][p_id], 'indri'])

In [375]:
# ranking_list_sorted = sorted(ranking_list, key=lambda x: (x[0],x[4]), reverse=True)
ranking_list_sorted = sorted(sorted(ranking_list, key = lambda x : x[4], reverse = True), key = lambda x : x[0])

In [376]:
ranking_list_sorted

[['0', '00', '267-15', 0, 10, 'indri'],
 ['0', '00', '267-14', 0, 7, 'indri'],
 ['0', '00', '267-46', 0, 6, 'indri'],
 ['0', '00', '267-2', 0, 6, 'indri'],
 ['0', '00', '267-12', 0, 6, 'indri'],
 ['0', '00', '267-24', 0, 5, 'indri'],
 ['0', '00', '267-27', 0, 5, 'indri'],
 ['0', '00', '267-22', 0, 5, 'indri'],
 ['0', '00', '267-47', 0, 5, 'indri'],
 ['0', '00', '267-0', 0, 5, 'indri'],
 ['0', '00', '267-13', 0, 5, 'indri'],
 ['0', '00', '267-16', 0, 5, 'indri'],
 ['0', '00', '267-60', 0, 4, 'indri'],
 ['0', '00', '267-29', 0, 4, 'indri'],
 ['0', '00', '267-41', 0, 4, 'indri'],
 ['0', '00', '267-6', 0, 4, 'indri'],
 ['0', '00', '267-59', 0, 4, 'indri'],
 ['0', '00', '267-11', 0, 4, 'indri'],
 ['0', '00', '267-10', 0, 4, 'indri'],
 ['0', '00', '267-17', 0, 4, 'indri'],
 ['0', '00', '267-55', 0, 4, 'indri'],
 ['0', '00', '267-42', 0, 3, 'indri'],
 ['0', '00', '267-48', 0, 3, 'indri'],
 ['0', '00', '267-62', 0, 3, 'indri'],
 ['0', '00', '267-64', 0, 3, 'indri'],
 ['0', '00', '267-49', 0, 3

In [87]:
len(ranking_list_sorted)

151862

In [377]:
cnt = 1
q_id = '0'
for i in range(len(ranking_list_sorted)):
    if cnt == 1:
        q_id = ranking_list_sorted[i][0]
        ranking_list_sorted[i][3] = cnt
    if ranking_list_sorted[i][0] == q_id:
        ranking_list_sorted[i][3] = cnt
        cnt = cnt + 1
    else:
        cnt = 1
        ranking_list_sorted[i][3] = cnt
        cnt = 2
        q_id = ranking_list_sorted[i][0]

In [378]:
ranking_list_sorted

[['0', '00', '267-15', 1, 10, 'indri'],
 ['0', '00', '267-14', 2, 7, 'indri'],
 ['0', '00', '267-46', 3, 6, 'indri'],
 ['0', '00', '267-2', 4, 6, 'indri'],
 ['0', '00', '267-12', 5, 6, 'indri'],
 ['0', '00', '267-24', 6, 5, 'indri'],
 ['0', '00', '267-27', 7, 5, 'indri'],
 ['0', '00', '267-22', 8, 5, 'indri'],
 ['0', '00', '267-47', 9, 5, 'indri'],
 ['0', '00', '267-0', 10, 5, 'indri'],
 ['0', '00', '267-13', 11, 5, 'indri'],
 ['0', '00', '267-16', 12, 5, 'indri'],
 ['0', '00', '267-60', 13, 4, 'indri'],
 ['0', '00', '267-29', 14, 4, 'indri'],
 ['0', '00', '267-41', 15, 4, 'indri'],
 ['0', '00', '267-6', 16, 4, 'indri'],
 ['0', '00', '267-59', 17, 4, 'indri'],
 ['0', '00', '267-11', 18, 4, 'indri'],
 ['0', '00', '267-10', 19, 4, 'indri'],
 ['0', '00', '267-17', 20, 4, 'indri'],
 ['0', '00', '267-55', 21, 4, 'indri'],
 ['0', '00', '267-42', 22, 3, 'indri'],
 ['0', '00', '267-48', 23, 3, 'indri'],
 ['0', '00', '267-62', 24, 3, 'indri'],
 ['0', '00', '267-64', 25, 3, 'indri'],
 ['0', '00'

### Write to file

In [379]:
with open("output.txt",'w') as resultFile:
    wr = csv.writer(resultFile, delimiter=' ')
    wr.writerows(ranking_list_sorted)

## Create index for every document

In [135]:
directory_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/index_doc/'
for doc_id in row_dict:
    directory = directory_root + doc_id.split('-')[0]
    if not os.path.exists(directory):
        os.makedirs(directory)

In [127]:
pre_pro_for_dict(corpus)

In [128]:
corpus['344']['1']

"Townspeople lived chartered town burgher , opposed serf lived village . Towns `` free '' , sense directly protected king emperor , feudal fief . [ citation needed ] Today process granting charter determined type government state question . In monarchy , charter royal charter given Crown state authority acting behalf Crown . In federation , granting charter jurisdiction lower level government state province . [ citation needed ] In Brazil , municipal corporation called municipios created mean local legislation state level , passing referendum vote affected population ."

## Create independent index for every document
### Group passages in different documents to different files for later indexing based on different documents

In [130]:
trec_list = []
for doc_k in corpus:
    passage = corpus[doc_k]
    for p_k in passage:
        trec_list.append('<DOC>\n' + 
                         '<DOCNO>' + doc_k + '-' + p_k + '</DOCNO>\n' + 
                         '<TEXT>' + passage[p_k] + '</TEXT>\n' + 
                         '</DOC>\n')
    with open('/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/trec/'+doc_k+'.txt', 'w') as single:
        for p in trec_list:
            single.write(p)
            single.write('\n')
    trec_list = []

### Split querys to different query files

In [217]:
def split_query(xml_path, out_root):
    with open(xml_path, 'r') as inp:
        st = inp.read().replace('\n', '')
    root = ET.fromstring(st)
    for child in root:
        rt = ET.Element('parameters')
        rt.append(child)
        ET.ElementTree(rt).write(out_root+child[1].text+'.txt')

In [327]:
query_list = ['/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query_dev.txt',
              '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query_dev_no.txt',
              '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query_test.txt',
              '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query_test_no.txt',
              '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query_train.txt']
out_root_list = ['/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/dev/',
                 '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/dev_no/',
                 '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/test/',
                 '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/test_no/',
                 '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/train/']
for i in range(5):
    split_query(query_list[i], out_root_list[i])

### Create index

In [134]:
index_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/index_doc/'
corpus_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/trec/'
for doc_k in corpus:
    index_loc = index_root + doc_k
    corpus_loc = corpus_root + doc_k + '.txt'
    args = 'IndriBuildIndex -index=' + index_loc + ' -corpus.path=' + corpus_loc + ' -corpus.class=trectext -memory=2g'
    s = sp.Popen(args, stdout=sp.PIPE, shell=True, encoding='utf-8')
    s.stdout.read()

### Set path root

In [357]:
out_dev_no_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/cv_no/'
out_dev_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/cv/'
out_test_no_root= '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/test_no/'
out_test_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/test/'
out_test_ql_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/ql_test/'
out_train_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/train/'
out_train_ql_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/ql_train/'

query_dev_no_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/dev_no/'
query_dev_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/dev/'
query_test_no_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/test_no/'
query_test_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/test/'
query_train_root = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/query/train/'

qrels_dev = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/qrels_dev.txt'
qrels_test = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/qrels_test.txt'
qrels_train = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/qrels_train.txt'

trec_eval = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/trec_eval.8.1/trec_eval'

### Query using bm25

In [323]:
def query_single_bm25(output_path, query_path, index_path, k1=1.2, b=7, k3=0.75, suffix=''):
    # query
    query_args = 'IndriRunQuery ' + query_path + ' -index=' + index_path + \
        ' -trecFormat=true -baseline=okapi' + \
        ',k1:' + str(k1) + 'b:' + str(b) + 'k3:' + str(k3) + ' > ' + output_path

    o = sp.Popen(query_args, stdout=sp.PIPE, shell=True, encoding='utf-8')
    o.stdout.read()

### Query using QL

In [324]:
def query_single_ql(output_path, query_path, index_path, suffix=''):
    # query
    query_args = 'IndriRunQuery ' + query_path + ' -index=' + index_path + \
        ' -trecFormat=true -rule=method:dirichlet,mu:2500 > ' + output_path
    o = sp.Popen(query_args, stdout=sp.PIPE, shell=True, encoding='utf-8')
    o.stdout.read()

### Combine query results into a single file for later evaluation

In [277]:
def combine_query(out_root):
    with open(out_root+'output.txt', 'w') as outfile:
        for fname in os.listdir(out_root):
            with open(out_root+fname, 'r') as infile:
                if fname.endswith("_out.txt"):
                    outfile.write(infile.read())

### Evaluation function using `trec_eval`

In [278]:
def eval_trec(output_path, trec_eval_path, grels_path):
    # evaluation
    eval_out = output_path + '_eval.txt'
    eval_args = trec_eval_path + ' ' + grels_path + ' ' + output_path + ' > ' + eval_out
    s = sp.Popen(eval_args, stdout=sp.PIPE, shell=True, encoding='utf-8')
    s.stdout.read()

Cross-validation

In [294]:
for k1 in np.arange(1.2,2.1,0.1):
    for q_k in rel_doc_id_dict:
        index_pt = index_root + rel_doc_id_dict[q_k]
        query_pt = query_dev_root + q_k + '.txt'
        output_pt = out_dev_root + q_k + '_out.txt'
        query_single_bm25(output_pt, query_pt, index_pt, k1=k1)
        
    combine_query(out_dev_root)
    eval_trec(out_dev_root+'output.txt', trec_eval, qrels_dev)
        
    with open(out_dev_root+'output.txt_eval.txt','r') as file:
        print(str(k1)+'\t'+file.readlines()[4])

1.2	map            	all	0.4581

1.3	map            	all	0.4578

1.4000000000000001	map            	all	0.4523

1.5000000000000002	map            	all	0.4514

1.6000000000000003	map            	all	0.4502

1.7000000000000004	map            	all	0.4468

1.8000000000000005	map            	all	0.4442

1.9000000000000006	map            	all	0.4428

2.000000000000001	map            	all	0.4392



Test with `k = 1.2`

In [490]:
for q_k in rel_doc_id_dict:
    index_pt = index_root + rel_doc_id_dict[q_k]
    query_pt = query_test_root + q_k + '.txt'
    output_pt = out_test_root + q_k + '_out.txt'
    query_single_bm25(output_pt, query_pt, index_pt)
    
combine_query(out_test_root)
eval_trec(out_test_root+'output.txt', trec_eval, qrels_test)

with open(out_test_root+'output.txt_eval.txt','r') as file:
    print(str(1.2)+'\t'+file.readlines()[4])

1.2	map            	all	0.4888



In [491]:
for q_k in rel_doc_id_dict:
    index_pt = index_root + rel_doc_id_dict[q_k]
    query_pt = query_test_root + q_k + '.txt'
    output_pt = out_test_ql_root + q_k + '_out.txt'
    query_single_ql(output_pt, query_pt, index_pt)
    
combine_query(out_test_ql_root)
eval_trec(out_test_ql_root+'output.txt', trec_eval, qrels_test)

with open(out_test_ql_root+'output.txt_eval.txt','r') as file:
    print('ql\t' + file.readlines()[4])

ql	map            	all	0.5384



## L2R

### Training data

In [409]:
ql_train_path = out_train_ql_root + 'output.txt'
train_path = out_train_root + 'output.txt'
y_path = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/train.tsv'

### Extract training data 

In [595]:
QA_trn = []
score_bm25_trn = []
score_ql_trn = []
true_QA_trn = set()

with open(y_path, newline='\n') as f:
    reader = csv.reader(f, delimiter='\t')
    next(reader, None)
    for row in reader:
        true_QA_trn.add(row[0]+'@'+row[2]+'-'+row[4])
        
rele_nr = len(true_QA_trn)

with open(train_path, newline='\n') as f:
    reader = csv.reader(f, delimiter=' ')
    cnt = 0
    for row in reader:
        if row[0]+'@'+row[2] not in true_QA_trn:
            cnt = cnt + 1
            if cnt > rele_nr:
                continue
        QA_trn.append(row[0]+'@'+row[2])
        score_bm25_trn.append(float(row[4]))
        
with open(ql_train_path, newline='\n') as f:
    reader = csv.reader(f, delimiter=' ')
    cnt = 0
    for row in reader:
        if row[0]+'@'+row[2] not in true_QA_trn:
            cnt = cnt + 1
            if cnt > rele_nr:
                continue
        score_ql_trn.append(float(row[4]))
    
        
X = np.array([score_bm25_trn, score_ql_trn]).reshape((len(QA_trn), 2))
y = np.zeros(len(QA_trn))

### Find `y = 1`

In [596]:
for i in range(len(QA_trn)):
    if QA_trn[i] in true_QA_trn:
        y[i] = 1

In [597]:
len(QA_trn), len(score_bm25_trn), len(score_ql_trn), len(true_QA_trn), y.shape, X.shape, y.sum(), X[1:4,:]

(4787,
 4787,
 4787,
 3332,
 (4787,),
 (4787, 2),
 1455.0,
 array([[-7.30402, -7.37346],
        [-7.61798, -7.70782],
        [-7.71915, -7.84381]]))

In [598]:
p_rank = PRank(n_iter=50)
p_rank.fit(X, y)

PRank(n_iter=50, random_state=None, shuffle=True)

In [599]:
y_out = p_rank.predict(X)

/Users/joy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [600]:
y_out.sum(), np.mean(y_out == y)

(178.0, 0.6642991435136829)

### Extract test data

In [601]:
bm25_tst_path = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/test/output.txt'
ql_tst_path = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/ql_test/output.txt'
tst_y_path = '/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/test.tsv'

QA_tst = []
score_bm25_tst = []
score_ql_tst = []
true_QA_tst = set()
NEW_tst = []

with open(tst_y_path, newline='\n') as f:
    reader = csv.reader(f, delimiter='\t')
    next(reader, None)
    for row in reader:
        true_QA_tst.add(row[0]+'@'+row[2]+'-'+row[4])

with open(bm25_tst_path, newline='\n') as f:
    reader = csv.reader(f, delimiter=' ')
    cnt = 0
    for row in reader:
        if row[0]+'@'+row[2] not in true_QA_tst:
            cnt = cnt + 1
            if cnt > len(true_QA_tst):
                continue
        NEW_tst.append([row[0],row[1],row[2],0,row[4],row[5]])
        QA_tst.append(row[0]+'@'+row[2])
        score_bm25_tst.append(float(row[4]))
        
with open(ql_tst_path, newline='\n') as f:
    reader = csv.reader(f, delimiter=' ')
    cnt = 0
    for row in reader:
        if row[0]+'@'+row[2] not in true_QA_tst:
            cnt = cnt + 1
            if cnt > len(true_QA_tst):
                continue
        score_ql_tst.append(float(row[4]))
        
X_tst = np.array([score_bm25_tst, score_ql_tst]).reshape((len(QA_tst), 2))
y_tst = np.zeros(len(QA_tst))

for i in range(len(QA_tst)):
    if QA_tst[i] in true_QA_tst:
        y_tst[i] = 1

In [602]:
X_tst.shape, y_tst.shape, y_tst.sum()

((601, 2), (601,), 185.0)

In [603]:
y_tst_out = p_rank.predict(X_tst)

/Users/joy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [604]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==0:
            TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN += 1

    return(TP, FP, TN, FN)

In [605]:
TF = perf_measure(y_tst, y_tst_out)
TF

(3, 24, 392, 182)

In [606]:
e = np.mean(y_tst == y_tst_out)
e

0.6572379367720466

### Extract output label to compare it with groudtruth and compute MAP

In [607]:
NEW = [NEW_tst[i] for i in range(len(y_tst_out)) if y_tst_out[i] == 1]
NEW

[['2129', 'Q0', '522-34', 0, '-1.52974', 'indri'],
 ['2129', 'Q0', '522-46', 0, '-1.54339', 'indri'],
 ['2129', 'Q0', '522-55', 0, '-1.55114', 'indri'],
 ['2129', 'Q0', '522-42', 0, '-1.58305', 'indri'],
 ['2129', 'Q0', '522-18', 0, '-1.58665', 'indri'],
 ['2129', 'Q0', '522-20', 0, '-1.5912', 'indri'],
 ['2129', 'Q0', '522-21', 0, '-1.60919', 'indri'],
 ['2139', 'Q0', '55-47', 0, '-0.741854', 'indri'],
 ['2139', 'Q0', '55-48', 0, '-0.748066', 'indri'],
 ['2139', 'Q0', '55-50', 0, '-1.16829', 'indri'],
 ['2139', 'Q0', '55-27', 0, '-1.80357', 'indri'],
 ['2139', 'Q0', '55-3', 0, '-1.88446', 'indri'],
 ['2139', 'Q0', '55-31', 0, '-1.8939', 'indri'],
 ['2139', 'Q0', '55-45', 0, '-1.91306', 'indri'],
 ['2139', 'Q0', '55-26', 0, '-1.92279', 'indri'],
 ['2139', 'Q0', '55-7', 0, '-1.93262', 'indri'],
 ['2139', 'Q0', '55-43', 0, '-1.9833', 'indri'],
 ['2139', 'Q0', '55-51', 0, '-2.00432', 'indri'],
 ['2139', 'Q0', '55-41', 0, '-2.03671', 'indri'],
 ['2139', 'Q0', '55-56', 0, '-2.04774', 'indri

### Change ranking

In [608]:
cnt = 1
q_id = '0'
for i in range(len(NEW)):
    if cnt == 1:
        q_id = NEW[i][0]
        NEW[i][3] = cnt
    if NEW[i][0] == q_id:
        NEW[i][3] = cnt
        cnt = cnt + 1
    else:
        cnt = 1
        NEW[i][3] = cnt
        cnt = 2
        q_id = NEW[i][0]
NEW

[['2129', 'Q0', '522-34', 1, '-1.52974', 'indri'],
 ['2129', 'Q0', '522-46', 2, '-1.54339', 'indri'],
 ['2129', 'Q0', '522-55', 3, '-1.55114', 'indri'],
 ['2129', 'Q0', '522-42', 4, '-1.58305', 'indri'],
 ['2129', 'Q0', '522-18', 5, '-1.58665', 'indri'],
 ['2129', 'Q0', '522-20', 6, '-1.5912', 'indri'],
 ['2129', 'Q0', '522-21', 7, '-1.60919', 'indri'],
 ['2139', 'Q0', '55-47', 1, '-0.741854', 'indri'],
 ['2139', 'Q0', '55-48', 2, '-0.748066', 'indri'],
 ['2139', 'Q0', '55-50', 3, '-1.16829', 'indri'],
 ['2139', 'Q0', '55-27', 4, '-1.80357', 'indri'],
 ['2139', 'Q0', '55-3', 5, '-1.88446', 'indri'],
 ['2139', 'Q0', '55-31', 6, '-1.8939', 'indri'],
 ['2139', 'Q0', '55-45', 7, '-1.91306', 'indri'],
 ['2139', 'Q0', '55-26', 8, '-1.92279', 'indri'],
 ['2139', 'Q0', '55-7', 9, '-1.93262', 'indri'],
 ['2139', 'Q0', '55-43', 10, '-1.9833', 'indri'],
 ['2139', 'Q0', '55-51', 11, '-2.00432', 'indri'],
 ['2139', 'Q0', '55-41', 12, '-2.03671', 'indri'],
 ['2139', 'Q0', '55-56', 13, '-2.04774', 'i

In [609]:
with open("/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/L2R/output.txt",'w') as resultFile:
    wr = csv.writer(resultFile, delimiter=' ')
    wr.writerows(NEW)

### Evaluate

In [610]:
eval_trec("/Users/joy/Documents/Study_in_Netherlands/IR/lab/WikiPassageQA/new_output/L2R/output.txt", 
          trec_eval, qrels_test)